# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:


# Dependencies and Setup
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
import geoviews as gv
import geoviews.tile_sources as gts
import holoviews as hv
from holoviews import opts
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")
import hvplot.pandas
import pandas as pd
import requests
import json
# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(r"C:\Users\User\OneDrive\Desktop\python-api-challenge\WeatherPy\output_data\cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lander,42.8330,-108.7307,15.54,37,0,2.06,US,1694942917
1,1,buchloe,48.0333,10.7333,23.28,75,64,0.89,DE,1694942918
2,2,corner brook,48.9667,-57.9484,13.09,90,55,5.78,CA,1694942679
3,3,klaksvik,62.2266,-6.5890,10.51,85,100,10.18,FO,1694942919
4,4,shingu,33.7333,135.9833,25.49,91,11,2.89,JP,1694942920


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
%%capture --no-display

import geoviews as gv
import geoviews.tile_sources as gvts
import pandas as pd

# Load _data_df from a CSV file
city_data_df = pd.read_csv(r"C:\Users\User\OneDrive\Desktop\python-api-challenge\WeatherPy\output_data\cities.csv")

# e
scatter_map = gv.Points(city_data_df, kdims=['Lng', 'Lat'], vdims=['City', 'Humidity'])

# Customize the appearance of the scatter map
scatter_map.opts(
    size=5,
    color='City',  
    cmap='Category20',  
    width=800,
    height=400,
    tools=['hover'],
    title="Cities and Humidity",
    bgcolor='white',
)


tiles = gvts.OSM()


final_map = tiles * scatter_map


final_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [54]:
import geoviews as gv
import geoviews.tile_sources as gvts
import pandas as pd

# Load city_data_df from a CSV file
city_data_df = pd.read_csv(r"C:\Users\User\OneDrive\Desktop\python-api-challenge\WeatherPy\output_data\cities.csv")

# Create a scatter map with city points
scatter_map = gv.Points(city_data_df, kdims=['Lng', 'Lat'], vdims=['City', 'Humidity'])

# Customize the appearance of the scatter map
scatter_map.opts(
    size=5,
    color='City',
    cmap='Category20',
    width=800,
    height=400,
    tools=['hover'],
    title="Cities and Humidity",
    bgcolor='white',
)

# Create a separate Text element to display city names outside the map
city_labels = gv.Text(city_data_df, kdims=['Lng', 'Lat'], vdims=['City'])

# Set the x, y, and text values for the city labels
city_labels.opts(
    text='City',  # Specify the column containing city names
    text_align='left',  # Adjust text alignment as needed
    text_baseline='middle',  # Adjust text baseline as needed
    xoffset=0.02,  # Adjust the horizontal offset as needed
    yoffset=0.02,  # Adjust the vertical offset as needed
)

# Combine the tile source, scatter map, and city labels
tiles = gvts.OSM()
final_map = tiles * scatter_map * city_labels

final_map


TypeError: Text.__init__() missing 2 required positional arguments: 'y' and 'text'

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Define your weather criteria
min_temp_celsius = 20.0  # Minimum temperature in Celsius
max_temp_celsius = 30.0  # Maximum temperature in Celsius
max_humidity = 50       # Maximum humidity percentage
max_wind_speed = 10     # Maximum wind speed in m/s

# Convert temperature from Kelvin to Celsius
city_data_df['Max Temp (C)'] = city_data_df['Max Temp']   # Convert Kelvin to Celsius

# Step 1: Narrow down the DataFrame based on criteria
filtered_cities_df = city_data_df[
    (city_data_df['Max Temp (C)'] >= min_temp_celsius) &
    (city_data_df['Max Temp (C)'] <= max_temp_celsius) &
    (city_data_df['Humidity'] < max_humidity) &
    (city_data_df['Wind Speed'] < max_wind_speed)
]

# Step 2: Drop any rows with null values
filtered_cities_df = filtered_cities_df.dropna()

# Rearrange the columns in the desired order
column_order = ['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country', 'Date']
filtered_cities_df = filtered_cities_df[column_order]

# Display the filtered DataFrame
filtered_cities_df



,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,dali,25.7000,100.1833,25.98,44,21,2.62,CN,1694942936
53,53,korla,41.7597,86.1469,25.18,21,36,3.08,CN,1694942945
57,57,bani walid,31.7566,13.9942,28.21,26,0,2.66,LY,1694942947
95,95,muswellbrook,-32.2667,150.9000,21.29,30,0,0.56,AU,1694942967
141,141,vernon,50.2581,-119.2691,20.80,42,100,0.51,CA,1694942711
155,155,hani,38.4167,40.4000,29.72,15,0,1.68,TR,1694942998
163,163,saynshand,44.8824,110.1163,20.18,28,72,5.47,MN,1694943003
165,165,ginir,7.1333,40.7000,26.20,35,80,3.94,ET,1694943004
167,167,ghanzi,-21.5667,21.7833,27.96,9,0,5.12,BW,1694943006
172,172,belyy yar,53.6039,91.3903,21.94,40,0,4.00,RU,1694943009


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Create a new DataFrame called hotel_df with selected columns from filtered_cities_df
hotel_df = filtered_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column "Hotel Name" to the DataFrame
hotel_df['Hotel Name'] = ''

# Display the hotel_df DataFrame with the specified column order
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Humidity', 'Hotel Name']]

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
36,dali,CN,25.7000,100.1833,44,
53,korla,CN,41.7597,86.1469,21,
57,bani walid,LY,31.7566,13.9942,26,
95,muswellbrook,AU,-32.2667,150.9000,30,
141,vernon,CA,50.2581,-119.2691,42,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [26]:
import requests
import pandas as pd

# API request and retrieve  hotel names
def get_hotel_name(lat, lon, params):
    try:
        response = requests.get("https://api.geoapify.com/v2/places", params=params)
        response.raise_for_status()
        data = response.json()
        hotels = data.get("features", [])
        
        if hotels:
            first_hotel = hotels[0]
            hotel_name = first_hotel["properties"].get("name", "No hotel name found")
        else:
            hotel_name = "No hotel found"
    except Exception as e:
        hotel_name = f"Error: {str(e)}"
    
    return hotel_name

# Set up  search for a hotel requred parameters
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key, 
    "limit": 20
}

print("Starting hotel search")

hotel_data = [] 

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lon = row["Lng"]

    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"

    hotel_name = get_hotel_name(lat, lon, params)

    # Append hotel data to the list
    hotel_data.append([
        row["City"],
        row["Country"],
        row["Lat"],
        row["Lng"],
        row["Humidity"],
        hotel_name
    ])

    # search results
    print(f"{row['City']} - nearest hotel: {hotel_name}")

# Create a DataFrame t
headers = ["City", "Country", "Lat", "Lng", "Humidity", "Hotel Name"]
hotel_df = pd.DataFrame(hotel_data, columns=headers)


display(hotel_df)


Starting hotel search
dali - nearest hotel: 风花雪月大酒店
korla - nearest hotel: Silver Star Hotel
bani walid - nearest hotel: فندق الزيتونة
muswellbrook - nearest hotel: No hotel found
vernon - nearest hotel: Okanagan Royal Park Inn
hani - nearest hotel: No hotel found
saynshand - nearest hotel: No hotel name found
ginir - nearest hotel: No hotel found
ghanzi - nearest hotel: Ghanzi farmhouse
belyy yar - nearest hotel: No hotel found
omurtag - nearest hotel: Замък Интер
damghan - nearest hotel: هتل جهانگردى دامغان
misratah - nearest hotel: الفندق السياحى
calipatria - nearest hotel: No hotel found
zhangatas - nearest hotel: Арман
alice springs - nearest hotel: Aurora Alice Springs
klyuchi - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
mashhad - nearest hotel: هتل تعاون
artemovskiy - nearest hotel: Счастье
altpinar - nearest hotel: No hotel found
caala - nearest hotel: No hotel found
rakovnik - nearest hotel: Rozmaryn
lyakhavichy - nearest hotel: Гасцініца Эдэльвейс
co

,City,Country,Lat,Lng,Humidity,Hotel Name
0,dali,CN,25.7000,100.1833,44,风花雪月大酒店
1,korla,CN,41.7597,86.1469,21,Silver Star Hotel
2,bani walid,LY,31.7566,13.9942,26,فندق الزيتونة
3,muswellbrook,AU,-32.2667,150.9000,30,No hotel found
4,vernon,CA,50.2581,-119.2691,42,Okanagan Royal Park Inn
5,hani,TR,38.4167,40.4000,15,No hotel found
6,saynshand,MN,44.8824,110.1163,28,No hotel name found
7,ginir,ET,7.1333,40.7000,35,No hotel found
8,ghanzi,BW,-21.5667,21.7833,9,Ghanzi farmhouse
9,belyy yar,RU,53.6039,91.3903,40,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [47]:
import geoviews as gv
import geoviews.tile_sources as gvts
from bokeh.models import HoverTool
import pandas as pd

# Create a scatter map 
hotel_scatter_map = gv.Points(hotel_df, kdims=['Lng', 'Lat'], vdims=['Hotel Name', 'City', 'Country', 'Humidity'] )


# Define additional dimensions for hover tooltips
hover = HoverTool()
hover.tooltips = [("City", "@City"), ("Country", "@Country"), ("Hotel Name", "@{Hotel Name}"), ("Lat", "@Lat"), ("Lng", "@Lng"), ("Humidity", "@Humidity")]

 
# Customize thotel scatter map
hotel_scatter_map.opts(
    size=5,
    color='Hotel Name',  
    cmap='Category20',
    width=800,
    height=400,
    tools=[hover], 
    title="Hotels",
    bgcolor='white',
    backend='bokeh'
)


tiles = gvts.OSM()

# Display the map
final2_map = tiles * hotel_scatter_map
final2_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City,Country,Humidity)